# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools


In [3]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [4]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v73'
version_v9 = 'v94'


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v75t', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v75t', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_v9, 'V9')
    ]

all_tpsets = {'DEF': 'dRC3d',
              'DEFCalib': 'NNDR Calib v1',
              'DEFNC': 'dRC3d + new Th',
              'HMvDR': 'HistoMaxC3d + dR(layer)',
              'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
              'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
              'EG': 'EG',
              'TkEG': 'TkEG',
              'TkEle': 'TkEle',
              'TkIsoEle': 'TkIsoEle',
              'L1Trk': 'L1Track'}


gen_part_selections: 8


Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v73.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v73.root does not exist

In [20]:
# %load settings.py
# === samples =====================================================
import python.selections as selections
import python.collections as collections

import pprint


samples = []

# samples = samples_nugunrates_V8
samples += samples_nugunrates_V9


sample = 'V9'

do_rate = True

# === TP ==========================================================
# tps = ['HMvDR', 'EG', 'TkEG', 'TkEle', 'TkIsoEle']
tps = ['HMvDR', 'EG', 'TkEG', 'TkEle', 'TkIsoEle',
       'HMvDRCalib', 'HMvDRcylind5Calib', 'HMvDRcylind2p5Calib', 'HMvDRshapeCalib']
# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3', 'EGq2EtaBC', 'EGq3EtaBC', 'EGq3EtaD']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']


for tp in tps:
    if tp not in tp_select.keys():
        tp_select[tp] = ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD']



# ==== GEN selections ===============================================

gen_select = ['GENEtaBC', 'GENEtaD']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']


event_manager = collections.EventManager()

all_tpsets = {}
for collection in event_manager.collections:
    all_tpsets[collection.name] = collection.label



# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})

import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


print selections.eg_rate_selections

--- TPs: 
[   'HMvDR',
    'EG',
    'TkEG',
    'TkEle',
    'TkIsoEle',
    'HMvDRCalib',
    'HMvDRcylind5Calib',
    'HMvDRcylind2p5Calib',
    'HMvDRshapeCalib']
--- TP selections:
{   'DEF': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'DEFNC': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'EG': ['EGq2', 'EGq3', 'EGq2EtaBC', 'EGq3EtaBC', 'EGq3EtaD'],
    'HMvDR': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRCalib': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRNC0': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRNC1': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRcylind2p5Calib': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRcylind5Calib': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRshapeCalib': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'L1Trk': ['all', 'Pt2', 'Pt10'],
    'TkEG': [   'EGq2M1P2',
                'EGq2M1P2S',
                'EGq3M1P10',
                'EGq2M1P5',
                'EGq2M1P10',
 

In [12]:
tpset_selections


{'EGq2': 'q2',
 'EGq2EtaBC': 'q2, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq2EtaBCD': 'q2, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq2EtaBCDM1': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07',
 'EGq2EtaBCDM1P10': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2EtaBCDM1P10S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2EtaBCDM1P2': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 2GeV',
 'EGq2EtaBCDM1P2S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 2GeV',
 'EGq2EtaBCDM1P5': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 5GeV',
 'EGq2EtaBCDM1P5S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 5GeV',
 'EGq2EtaBCM1': 'q2, 1.52 < |#eta^{L1}| <= 2.4, |#Delta#phi| <0.08 & #DeltaR < 0.07',
 'EGq2EtaBCM1P10': 'q2, 1.52 < |#eta^{L1}| <= 2.4,

In [13]:
gen_selections

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [14]:
tpset_selections

{'EGq2': 'q2',
 'EGq2EtaBC': 'q2, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq2EtaBCD': 'q2, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq2EtaBCDM1': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07',
 'EGq2EtaBCDM1P10': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2EtaBCDM1P10S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2EtaBCDM1P2': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 2GeV',
 'EGq2EtaBCDM1P2S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 2GeV',
 'EGq2EtaBCDM1P5': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 5GeV',
 'EGq2EtaBCDM1P5S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 5GeV',
 'EGq2EtaBCM1': 'q2, 1.52 < |#eta^{L1}| <= 2.4, |#Delta#phi| <0.08 & #DeltaR < 0.07',
 'EGq2EtaBCM1P10': 'q2, 1.52 < |#eta^{L1}| <= 2.4,

In [15]:
from python.plotters import rate_plotters, eg_rate_plotters
histo_primitives = pd.DataFrame() 

for plotter in rate_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
for plotter in eg_rate_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
print histo_primitives.data.unique()

['HMvDR' 'HMvDRCalib' 'HMvDRcylind5Calib' 'HMvDRcylind2p5Calib'
 'HMvDRshapeCalib' 'EG' 'TkEG' 'TkEle' 'TkIsoEle']


In [21]:
%%time
hplot = HPlot(samples, tpsets, tpset_selections, [])

for tp in tps:
    print tp
    print histo_primitives[histo_primitives.data == tp].data_sel.unique()
    print gen_selections
    print samples
    hplot.cache_histo(classtype=histos.RateHistos, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=None)


HMvDR
['all' 'EtaD' 'EtaBC' 'EtaBCD' 'Em' 'EmEtaD' 'EmEtaBC' 'EmEtaBCD']
{'GENPt40': 'p_{T}^{GEN}>=40GeV', 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8', 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4', 'GENPt30': 'p_{T}^{GEN}>=30GeV', 'GENPt20': 'p_{T}^{GEN}>=20GeV', 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8', 'GENPt10': 'p_{T}^{GEN}>=10GeV', 'nomatch': '', 'GEN': ''}
[<drawingTools.Sample instance at 0x11fe77cb0>]
<drawingTools.Sample instance at 0x11fe77cb0>
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR all None
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR EtaD None
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR EtaBC None
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR EtaBCD None
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR Em None
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR EmEtaD None
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR EmEtaBC None
<drawingTools.Sample instance at 0x11fe77cb0> HMvDR EmEtaBCD None
EG
['EGq2' 'EGq2EtaD' 'EGq2EtaBC' 'EGq2EtaBC

<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p2 None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p2EtaD None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p2EtaBC None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p2EtaBCD None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p3 None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p3EtaD None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p3EtaBC None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq2Iso0p3EtaBCD None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq3 None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq3EtaD None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq3EtaBC None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq3EtaBCD None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle EGq3Iso0p2 None
<drawingTools.Sample instance at 0x11fe77cb0> TkIsoEle E

In [22]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V9,PU200,HMvDR,all,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1211f4dd0>
1,V9,PU200,HMvDR,EtaD,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x121063290>
2,V9,PU200,HMvDR,EtaBC,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1210f30e0>
3,V9,PU200,HMvDR,EtaBCD,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x121044488>
4,V9,PU200,HMvDR,Em,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1211f3950>
5,V9,PU200,HMvDR,EmEtaD,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1210f3950>
6,V9,PU200,HMvDR,EmEtaBC,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x12103da28>
7,V9,PU200,HMvDR,EmEtaBCD,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1210b1680>
8,V9,PU200,EG,EGq2,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1210b78c0>
9,V9,PU200,EG,EGq2EtaD,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x12103b878>


In [23]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        print tp,tp_sel
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)


HMvDR all
normalize to 31000
HMvDR EtaBC
normalize to 31000
HMvDR Em
normalize to 31000
HMvDR EmEtaBC
normalize to 31000
HMvDR EmEtaD
normalize to 31000
EG EGq2
normalize to 31000
EG EGq3
normalize to 31000
EG EGq2EtaBC
normalize to 31000
EG EGq3EtaBC
normalize to 31000
EG EGq3EtaD
normalize to 31000
TkEG EGq2M1P2
normalize to 31000
TkEG EGq2M1P2S
normalize to 31000
TkEG EGq3M1P10
normalize to 31000
TkEG EGq2M1P5
normalize to 31000
TkEG EGq2M1P10
normalize to 31000
TkEG EGq3M1
normalize to 31000
TkEG EGq3M1P5S
normalize to 31000
TkEG EGq3
normalize to 31000
TkEG EGq2
normalize to 31000
TkEG EGq3M1P2S
normalize to 31000
TkEG EGq2M1P5S
normalize to 31000
TkEG EGq2M1P10S
normalize to 31000
TkEG EGq3M1P2
normalize to 31000
TkEG EGq2M1
normalize to 31000
TkEG EGq3M1P5
normalize to 31000
TkEG EGq3M1P10S
normalize to 31000
TkEle EGq2
normalize to 31000
TkEle EGq3
normalize to 31000
TkEle EGq2Iso0p2
normalize to 31000
TkEle EGq3Iso0p2
normalize to 31000
TkEle EGq2Iso0p3
normalize to 31000
TkEl

#### 3D clusters matched to GEN

In [24]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_norm for his in hsets], labels, text=text)
         

In [25]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_pt for his in hsets], labels, text=text, logy=True)
         

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [26]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDR', 'HMvDRCalib'], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

<ROOT.TCanvas object ("c_d2e0d6") at 0x7fac3cbb2950>

In [33]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib', 'HMvDRcylind5Calib', 'HMvDRcylind2p5Calib', 'HMvDRshapeCalib'], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

<ROOT.TCanvas object ("c_2b1751") at 0x7fac3cc3b710>

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [34]:
hsets_new = []
labels_new = []
    
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEle'], 'EGq3EtaBC', None)
hsets_new.append(hsets[0])
labels_new.append("TkEle")

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEG'], 'EGq3EtaBCM1P2', None)
for hist in hsets:
    hist.normalize(31000)
hsets_new.append(hsets[0])
labels_new.append("p_{T}^{trk} > 2GeV")


hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEG'], 'EGq3EtaBCM1P5', None)
for hist in hsets:
    hist.normalize(31000)
hsets_new.append(hsets[0])
labels_new.append("p_{T}^{trk} > 5GeV")



hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEG'], 'EGq3EtaBCM1P10', None)
for hist in hsets:
    hist.normalize(31000)
hsets_new.append(hsets[0])
labels_new.append("p_{T}^{trk} > 10GeV")


draw([his.h_pt for his in hsets_new], labels_new, text=text, logy=True)


normalize to 31000
normalize to 31000
normalize to 31000


<ROOT.TCanvas object ("c_e2d54c") at 0x7fac3cdbefb0>

In [48]:
for tp_sel in tp_select['DEF']:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V8', 'V9'], 'PU200', ['DEF','HMvDR'], tp_sel, None)
    for hist in hsets:
        hist.normalize(31000)
    draw([his.h_pt for his in hsets], labels, text=text, logy=True)
     

normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [38]:
for tp in tps:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', tp, ['EmEtaABC', 'EmEtaBC'], None)
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    hist_list = [his.h_pt for his in hsets]
    draw(hist_list, labels, text=text, logy=True, do_ratio=True)
#     hRatio = hist_list[1].Clone(uuid.uuid4().hex[:6])
#     stuff.append(hRatio)
#     hRatio.Divide(hist_list[0])
#     drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

c_732713 c_732713 800 800
0 
[<ROOT.TH1F object ("DEF_EmEtaBC_pt") at 0x7fb57d845d10>]


IndexError: list index out of range

In [15]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', ['DEF','DEFCalib'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
drawSame(hist_list, labels, text=text, logy=True)
hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
stuff.append(hRatio)
hRatio.Divide(hist_list[1])
drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [39]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', ['DEF','DEFMerged'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
draw(hist_list, labels, text=text, logy=True, do_ratio = True)
# hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
# stuff.append(hRatio)
# hRatio.Divide(hist_list[1])
# drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

c_67fe02 c_67fe02 800 800
0 
1 same,
[<ROOT.TH1F object ("DEFMerged_EmEtaBC_pt") at 0x7fb57e908670>, <ROOT.TH1F object ("DEF_EmEtaBC_pt") at 0x7fb57d845d10>]
0.623742461205 36563.409375


<ROOT.TCanvas object ("c_67fe02") at 0x7fb57e915330>

### Comparisono baseline vs Manos EGID

In [18]:
for tp in ['DEF']:
    for etaregion in [['EmEtaBC', 'Emv1EtaBC']]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', tp, etaregion, None)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        hist_list = [his.h_pt for his in hsets]
        drawSame(hist_list, labels, text=text, logy=True)
        hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
        stuff.append(hRatio)
        hRatio.Divide(hist_list[1])
        drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [19]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        # drawSame([his.h_ptVabseta for his in hsets], labels, text=text)
  

In [20]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 11) for his in hsets], labels, text=text+', p_{T}^{L1}>10GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')



In [21]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 21) for his in hsets], labels, text=text+', p_{T}^{L1}>20GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [22]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 31) for his in hsets], labels, text=text+', p_{T}^{L1}>30GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [23]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 41) for his in hsets], labels, text=text+', p_{T}^{L1}>40GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')
